In [2]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
output=pd.read_csv('data_with_internal_info.csv')
rows, columns = output.shape
print("Number of rows:", rows)
print("Number of columns:", columns)

C:\Users\nazan\AppData\Local\Temp\ipykernel_33532\331495582.py:2: DtypeWarning: Columns (12,13,14,15,16,17,24,29,35) have mixed types. Specify dtype option on import or set low_memory=False.
  output=pd.read_csv('data_with_internal_info.csv')


Number of rows: 235277
Number of columns: 60


In [4]:
import string
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sentence_transformers import SentenceTransformer

nltk.download("wordnet")
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

_stopwords = set(stopwords.words("english"))
_lemmatizer = WordNetLemmatizer()

def text_prep(text_list):
    cleaned = []
    table = str.maketrans('', '', string.punctuation)
    for doc in text_list:
        doc = str(doc).lower().translate(table)
        doc = doc.replace('\n', ' ')
        doc = re.sub(r'[^a-z0-9._\s]', '', doc)
        doc = re.sub(r'\s+', ' ', doc).strip()
        tokens = [tok for tok in doc.split() if tok not in _stopwords and len(tok) > 2]
        lemmas = [_lemmatizer.lemmatize(tok) for tok in tokens]
        cleaned.append(" ".join(lemmas))
    return cleaned


columns_to_combine = [
    'Job Title (en)', 'Job Title (nl)', 'Level Title', 'Function goal', 
    'Key result areas: result area (1)', 'Key result areas: result area (2)',
    'Key result areas: result area (3)', 'Key result areas: result area (4)',
    'Key result areas: result area (5)', 'Key result areas: result area (6)',
    'Key result areas: result area (7)', 'Key result areas: result area (8)',
    'Key result areas: result area (9)', 'Key result areas: result area (10)',
    'Key result areas: result area (11)', 'Specify the budget amounts.', 
    'Diploma Category', 'Speciality', 'Required experience', 'Innovation', 
    'Row 4 - Column 1', 'Row 5 - Column 1',
    'Row 6 - Column 1', 'Row 7 - Column 1', 'Row 8 - Column 1',
    'Internal Job', 'Internal Job Grade', 'Department'
]

output[columns_to_combine] = output[columns_to_combine].astype(str)
output['Yearly Gross Base Salary'] = pd.to_numeric(output['Yearly Gross Base Salary'], errors='coerce')


text_combined = output[columns_to_combine].agg(' '.join, axis=1).tolist()
text_cleaned = text_prep(text_combined)


embedder = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
X = embedder.encode(text_cleaned, show_progress_bar=True)
X = pd.DataFrame(X, index=output.index)

y = output['Yearly Gross Base Salary'].fillna(output['Yearly Gross Base Salary'].mean())


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)


rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    random_state=11,
    n_jobs=-1
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print("--- Salary Prediction ---")
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred) ** 0.5)
print("R2:", r2_score(y_test, y_pred))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred))


c:\Users\nazan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nazan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nazan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Batches: 100%|██████████| 7353/7353 [1:19:21<00:00,  1.54it/s]


--- Salary Prediction ---
MAE: 1870.5103037935796
RMSE: 6904.304761217076
R2: 0.06198464005923776
MAPE: 0.028626305816052838


In [6]:
#Random Forest fine-tuning with GridSearchCV
from sklearn.model_selection import GridSearchCV
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth':    [ 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf':  [1, 2],
    'max_features': ['log2', 'sqrt']
}

grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=2)


grid_search_rf.fit(X_train, y_train)

best_rf = grid_search_rf.best_estimator_
print("Best hyperparameters for RandomForest:", grid_search_rf.best_params_)


y_pred_rf = best_rf.predict(X_test)

print("\n Evaluation for Salary prediction (Random Forest with GridSearch) ")
print("  MAE: ", mean_absolute_error(y_test, y_pred_rf))
print("  RMSE:", mean_squared_error(y_test, y_pred_rf) ** 0.5)
print("  R²:  ", r2_score(y_test, y_pred_rf))
print("  MAPE:", mean_absolute_percentage_error(y_test, y_pred_rf))

Fitting 3 folds for each of 32 candidates, totalling 96 fits
Best hyperparameters for RandomForest: {'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}

 Evaluation for Salary prediction (Random Forest with GridSearch) 
  MAE:  1735.0538317470498
  RMSE: 6931.8861685492775
  R²:   0.05447527855402068
  MAPE: 0.026693830858119365


In [7]:
!pip install xgboost


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import sys
!{sys.executable} -m pip install xgboost


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:

import xgboost as xgb
model_xgb = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.05,
    objective='reg:squarederror',
    random_state=11
)
model_xgb.fit(X_train, y_train)

# Predictions
y_pred_xgb = model_xgb.predict(X_test)

# Evaluation
print("\n--- Evaluation for Salary prediction (XGBoost with Embeddings(base model)) ---")
print("  MAE:", mean_absolute_error(y_test, y_pred_xgb))
print("  RMSE:", mean_squared_error(y_test, y_pred_xgb) ** 0.5)
print("  R²:", r2_score(y_test, y_pred_xgb))
print("  MAPE:", mean_absolute_percentage_error(y_test, y_pred_xgb))


--- Evaluation for Salary prediction (XGBoost with Embeddings(base model)) ---
  MAE: 1993.1919064757512
  RMSE: 6883.414163460658
  R²: 0.0676524245933251
  MAPE: 0.03036814999341633


In [10]:
#fine-tuning XGBoost with GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 1.0]
}
grid_search = GridSearchCV(
    model_xgb,
    param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1,
    verbose=2
    
)
grid_search.fit(X_train, y_train)
print("Best hyperparameters for XGBoost:", grid_search.best_params_)
best_xgb = grid_search.best_estimator_

y_true = y_test.values.flatten()
y_pred = best_xgb.predict(X_test)

print("\nEvaluation for Salary with XGBoost (tuned model)")
print("  MAE: ", mean_absolute_error(y_true, y_pred))
print("  RMSE:", mean_squared_error(y_true, y_pred)**0.5)
print("  R²:  ", r2_score(y_true, y_pred))
print("  MAPE:", mean_absolute_percentage_error(y_true, y_pred))

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   9.1s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   9.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   8.7s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   8.9s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   9.4s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   9.6s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.6; total time=  18.8s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.6; total time=  18.9s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.6; total time=  18.8s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, subsample=1.0; total time=  19.5s

In [11]:
import joblib
import os

save_dir = "models/salary"
os.makedirs(save_dir, exist_ok=True)

# 1. Save XGBoost model
best_xgb.save_model(f"{save_dir}/salary_xgboost.json")

# 2. Save the SentenceTransformer embedder
embedder.save(f"{save_dir}/embedding_model")

# 3. Save metadata (columns list)
joblib.dump(columns_to_combine, f"{save_dir}/columns_to_combine.pkl")

print("Salary model saved successfully!")

Salary model saved successfully!
